# CCD calculations

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## CCD calculations on H2O // STO-3G

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz")
N = molecule.numberOfElectrons()
N_P = N // 2

print(molecule)

Number of electrons: 10
O  (0, -0.143226, 0)
H  (1.63804, 1.13655, -0)
H  (-1.63804, 1.13655, -0)



The first task is to generate the RHF reference, so we'll do an RHF SCF calculation.

In [3]:
r_spinor_basis = gqcpy.RSpinorBasis(molecule, "STO-3G")
K = r_spinor_basis.numberOfSpatialOrbitals()

S = r_spinor_basis.quantizeOverlapOperator().parameters()

In [4]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(r_spinor_basis, molecule)  # 'sq' for 'second-quantized'

The current Hamiltionian is expressed in the non-orthogonal AO basis. This is exactly what we need to start an RHF calculation.

In [5]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, sq_hamiltonian, S)
solver = gqcpy.RHFSCFSolver.DIIS()

objective = gqcpy.DiagonalRHFFockMatrixObjective(sq_hamiltonian)  # use the default threshold of 1.0e-08

Using this objective makes sure that the optimized expansion coefficients yield a diagonal Fock matrix, so they will represent the canonical RHF spin-orbital basis.

In [6]:
rhf_qc_structure = gqcpy.RHF.optimize(objective, solver, environment)

In [7]:
rhf_energy = rhf_qc_structure.groundStateEnergy() + gqcpy.Operator.NuclearRepulsion(molecule).value()
print(rhf_energy)

-74.94207992809503


This is very much in line with `crawdad`'s results: `-74.942079928192`.

In [8]:
rhf_parameters = rhf_qc_structure.groundStateParameters()
C = rhf_parameters.coefficientMatrix()

In [9]:
r_spinor_basis.transform(C)  # now represents a restricted spin-orbital basis using the canonical RHF MOs

Before continuing with the CCSD specifics, we must first prepare the molecular Hamiltonian in the correct spinor basis. Since we have implemented CCSD using general spinors, we should use a `GSpinorBasis`.

In [10]:
g_spinor_basis = gqcpy.GSpinorBasis.FromRestricted(r_spinor_basis)  # represents a general spinor basis, based off the restricted canonical RHF MOs

M = g_spinor_basis.numberOfSpinors()
ghf_onv = gqcpy.SpinUnresolvedONV.GHF(M, N, rhf_parameters.spinOrbitalEnergiesBlocked())

orbital_space = ghf_onv.orbitalSpace()
print(orbital_space.description())

An orbital space with 14 orbitals.

	The occupied orbital indices are: 0 1 2 3 4 7 8 9 10 11 
	The active orbital indices are: 
	The virtual orbital indices are: 5 6 12 13 


CCSD is also implemented as a `QCMethod`. This means that we'll have to set up an environment and a particular solver.

In [11]:
sq_hamiltonian = gqcpy.SQHamiltonian.Molecular(g_spinor_basis, molecule)

In [12]:
solver = gqcpy.CCDSolver.Plain()
environment = gqcpy.CCSDEnvironment.PerturbativeCCD(sq_hamiltonian, orbital_space)

Let's now proceed by actually optimizing the CCD wave function model. This may take a while if `gqcp` is built in Debug mode!

In [13]:
ccd_qc_structure = gqcpy.CCD.optimize(solver, environment)

In [14]:
ccd_correlation_energy = ccd_qc_structure.groundStateEnergy()
print(ccd_correlation_energy)

-0.07015048702156702


In [18]:
def ccd0(environment):
    t2_amplitudes = environment.t2_amplitudes[-1].asTensor()
    
    dim = t2_amplitudes.shape
    new_t2_amplitudes = np.zeros(dim)
    
#    for i in range(dim[0]):
#        for j in range(dim[1]):
#            for a in range(dim[2]):
#                for b in range(dim[3]):
#                    new_t2_amplitudes[i,j,a,b] = 0.5*t2_amplitudes[i,j,a,b]*t2_amplitudes[i,j,b,a]
                    
    new_t2_amplitudes = 0.5 * t2_amplitudes * t2_amplitudes.transpose(0,1,3,2)
    environment.replace_current_t2_amplitudes(new_t2_amplitudes, dim[0], dim[2])

In [19]:
CCD0_step = gqcpy.FunctionalStep_CCSDEnvironment(ccd0, description="Function that includes the CCD0 step in the CCD algorithm.")

In [20]:
ccd0(environment)

/Users/daria/opt/anaconda3/envs/gqcp_dev/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in multiply
  del sys.path[0]


TypeError: replace_current_t2_amplitudes(): incompatible function arguments. The following argument types are supported:
    1. (self: gqcpy.CCSDEnvironment, arg0: GQCP::T2Amplitudes<double>) -> None
    2. (self: gqcpy.CCSDEnvironment, arg0: Eigen::Tensor<double, 4, 0, long>, arg1: int, arg2: int) -> None

Invoked with: <gqcpy.CCSDEnvironment object at 0x11effee70>, array([[[[ 0.000e+000,  1.496e-058,  0.000e+000, -8.049e-041],
         [ 1.496e-058,  0.000e+000,  0.000e+000, -0.000e+000],
         [ 0.000e+000,  0.000e+000,  1.826e-043, -0.000e+000],
         [-8.049e-041, -0.000e+000, -0.000e+000,  0.000e+000]],

        [[ 3.138e-073,  0.000e+000,  0.000e+000, -0.000e+000],
         [ 0.000e+000,  0.000e+000,  0.000e+000,  0.000e+000],
         [ 0.000e+000,  0.000e+000,  0.000e+000,  0.000e+000],
         [-0.000e+000,  0.000e+000,  0.000e+000,  0.000e+000]],

        [[ 0.000e+000,  0.000e+000,  0.000e+000,  0.000e+000],
         [ 0.000e+000,  0.000e+000,  0.000e+000,  0.000e+00